#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable


Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18836 sha256=cfa121f678ff7ebdde89ecdcb16b7b4714c9aba5335fec0c01971b4e6f4432ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-9u_tfvkq/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-10-02 16:12:52.053845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-02 16:12:52.087995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-02 16:12:52.088514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 16:12:52.571645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [5]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [6]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
json_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.h5'
kerasNLP_model = Boston311KerasNLP()
kerasNLP_model.load( json_file, model_file)
kerasNLP_model.predict_date_rang = {'start':'2023-08-27', 'end':tomorrows_datestring}


In [7]:
data = kerasNLP_model.load_data( 'predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0]


In [8]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [9]:
clean_data.head()

,case_enquiry_id,type_Abandoned Bicycle,type_Abandoned Building,type_Abandoned Vehicles,type_Abandoned Vehicles - Private Tow,type_Aircraft Noise Disturbance,type_Alert Boston,type_Animal Found,type_Animal Generic Request,type_Animal Lost,...,ward_number_20,ward_number_21,ward_number_22,ward_number_3,ward_number_4,ward_number_5,ward_number_6,ward_number_7,ward_number_8,ward_number_9
0,101005013792,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,101005013793,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,101005013794,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,101005013795,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,101005013796,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [10]:
mydata = clean_data

In [11]:
mydata['case_enquiry_id']

0        101005013792
1        101005013793
2        101005013794
3        101005013795
4        101005013796
             ...     
29776    101005079312
29777    101005079313
29778    101005079315
29779    101005079316
29780    101005079318
Name: case_enquiry_id, Length: 29781, dtype: object

In [12]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [13]:
#concatenate the two dataframes and reindex
df = X

In [14]:
df.shape

(5728, 3)

In [15]:

# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)

In [16]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [17]:
df = df[is_numeric]

In [18]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [19]:
df.shape

(5728, 257)

In [20]:
mydata.shape

(29781, 436)

In [21]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [22]:
new_mydata.shape

(2003, 692)

In [23]:

df = new_mydata

In [24]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [25]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id                           object
type_Abandoned Bicycle                   float64
type_Abandoned Building                  float64
type_Abandoned Vehicles                  float64
type_Abandoned Vehicles - Private Tow    float64
                                          ...   
pooled_123                               float64
pooled_124                               float64
pooled_125                               float64
pooled_126                               float64
pooled_127                               float64
Length: 692, dtype: object


In [26]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [27]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [28]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

63/63 [==============================] - 0s 505us/step


In [29]:
predicted_labels = np.argmax(predictions, axis=1)

In [30]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':predicted_labels})

In [31]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [32]:
predictions_df.head(20)

,case_enquiry_id,prediction
0,101005013821,4+ months
1,101005013959,4+ months
2,101005014044,4+ months
3,101005014077,4+ months
4,101005014081,4+ months
5,101005014096,0-12 hours
6,101005014106,4+ months
7,101005014111,4-7 days
8,101005014166,4+ months
9,101005014174,4+ months


,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours
200358,101005013792,2023-08-26 00:37:05,2023-08-31 04:30:00,2023-08-26 09:01:50,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 13:01:50...,Poor Conditions of Property,Public Works Department,Code Enforcement,...,62 Kirkwood Rd,2135.0,42.340940,-71.155951,0101000020E61000001693AA18FBC951C0A65014E9A32B...,Citizens Connect App,0 days 08:24:45,1,22,8.412500
200359,101005013793,2023-08-26 00:38:25,2023-08-31 04:30:00,2023-08-26 09:06:05,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 13:06:05...,Poor Conditions of Property,Public Works Department,Code Enforcement,...,9 Radnor Rd,2135.0,42.340490,-71.158741,0101000020E6100000D1C9CDCE28CA51C041983D2A952B...,Citizens Connect App,0 days 08:27:40,1,22,8.461111
200360,101005013794,2023-08-26 00:39:33,2023-08-31 04:30:00,2023-08-26 09:17:45,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 13:17:45...,Poor Conditions of Property,Public Works Department,Code Enforcement,...,5 Radnor Rd,2135.0,42.340530,-71.158911,0101000020E6100000BBE6D6972BCA51C09180C979962B...,Citizens Connect App,0 days 08:38:12,1,22,8.636667
200361,101005013795,2023-08-26 00:42:01,2023-08-29 04:30:00,2023-08-28 06:28:55,ONTIME,Closed,Case Closed. Closed date : 2023-08-28 10:28:55...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,20 Westcott St,2124.0,42.292770,-71.078621,0101000020E6100000B74EDC1E08C551C0FF2C05797925...,Citizens Connect App,2 days 05:46:54,1,14,53.781667
200362,101005013796,2023-08-26 00:44:26,2023-08-29 04:30:00,2023-08-28 06:29:09,ONTIME,Closed,Case Closed. Closed date : 2023-08-28 10:29:09...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,10 Radnor Rd,2135.0,42.340870,-71.158681,0101000020E61000006D8824D327CA51C0F2F0E79DA12B...,Citizens Connect App,2 days 05:44:43,1,22,53.745278
200363,101005013797,2023-08-26 00:45:38,2023-08-29 04:30:00,2023-08-28 06:29:27,ONTIME,Closed,Case Closed. Closed date : 2023-08-28 10:29:27...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,19 Westcott St,2124.0,42.292880,-71.079001,0101000020E61000006051B3580EC551C0A5C8C4137D25...,Citizens Connect App,2 days 05:43:49,1,14,53.730278
200364,101005013798,2023-08-26 00:47:45,2023-08-30 04:30:00,2023-08-26 08:05:56,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 12:05:56...,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,15 Greycliff Rd,2135.0,42.340750,-71.160931,0101000020E6100000904859B04CCA51C0EB234CAF9D2B...,Citizens Connect App,0 days 07:18:11,1,22,7.303056
200365,101005013799,2023-08-26 00:48:27,2023-08-30 04:30:00,2023-08-26 08:16:56,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 12:16:56...,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,35 Greycliff Rd,2135.0,42.341460,-71.160521,0101000020E6100000246BACF845CA51C04FCD34F3B42B...,Citizens Connect App,0 days 07:28:29,1,22,7.474722
200366,101005013800,2023-08-26 01:39:00,2023-08-29 04:30:00,2023-08-27 02:08:01,ONTIME,Closed,Case Closed. Closed date : Sun Aug 27 06:08:01...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,190 Adams St,2122.0,42.302560,-71.060131,0101000020E61000007203EA2DD9C351C028114545BA26...,Citizens Connect App,1 days 00:29:01,1,15,24.483611
200367,101005013801,2023-08-26 01:42:21,2023-08-30 04:30:00,2023-08-26 04:05:15,ONTIME,Closed,Case Closed. Closed date : Sat Aug 26 08:05:15...,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,71-73 Chandler St,2116.0,42.346790,-71.072244,0101000020E6100000F5E0D8A59FC451C064C786A1632C...,Citizens Connect App,0 days 02:22:54,1,5,2.381667


In [48]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [49]:
import pandas as pd

model_name = '20230925_143704_Boston311KerasNLP'
ml_model_date = '2023-09-25'

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

In [36]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time'], axis=1).copy()

In [46]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [47]:
all_model_predictions['ml_model_name'] = model_name

In [50]:
all_model_predictions['prediction_date'] = today_datestring

In [51]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = False
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER






cp: cannot create regular file '/home/briarmoss/Documents/BODC-DEI-site/database/seeders': No such file or directory
cp: cannot create regular file '/home/briarmoss/Documents/BODC-DEI-site/database/seeders': No such file or directory
cp: cannot create regular file '/home/briarmoss/Documents/BODC-DEI-site/database/seeders': No such file or directory
cp: cannot create regular file '/home/briarmoss/Documents/BODC-DEI-site/database/seeders': No such file or directory
Executing: scp -P 65002 2023-10-02_17-04-54_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders


Host key verification failed.
lost connection


Executing: scp -P 65002 2023-10-02_17-04-54_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders


Host key verification failed.
lost connection


Executing: scp -P 65002 2023-10-02_17-04-54_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders


Host key verification failed.
lost connection


Executing: scp -P 65002 2023-10-02_17-04-54_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders


Host key verification failed.
lost connection


65002

In [ ]:
# %%
!ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

# %%
!ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

# %%